In [58]:
# dependencies
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from torch.utils.tensorboard import SummaryWriter
import numpy as np
import datetime
import pickle
import pandas as pd

In [59]:
# Load the dataset
data = pd.read_csv('Churn_Modelling.csv')
df = pd.DataFrame(data)
df.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [60]:
# remove unncessary columns
df = df.drop(columns=['CustomerId','RowNumber','Surname'], axis=1)


In [61]:
# Use Label Encoding for Gender selection and OneHotEncoding for Geography

from sklearn.preprocessing import LabelEncoder, OneHotEncoder

labelencoder_gender = LabelEncoder()
df['Gender'] = labelencoder_gender.fit_transform(df['Gender'])


# Initialize OneHotEncoder
onehotencoder_geography = OneHotEncoder()

# Fit and transform the 'Geography' column
geography_encoded = onehotencoder_geography.fit_transform(df[['Geography']]).toarray()

# Convert the encoded data into a DataFrame with proper column names
geography_encoded_df = pd.DataFrame(
    geography_encoded, 
    columns=onehotencoder_geography.get_feature_names_out(['Geography'])
)

# Concatenate the new DataFrame with the original one
df = pd.concat([df, geography_encoded_df], axis=1)

# Drop the original 'Geography' column
df = df.drop(columns=['Geography'])



In [62]:
df.head()

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain
0,619,0,42,2,0.00,1,1,1,101348.88,1,1.0,0.0,0.0
1,608,0,41,1,83807.86,1,0,1,112542.58,0,0.0,0.0,1.0
2,502,0,42,8,159660.80,3,1,0,113931.57,1,1.0,0.0,0.0
3,699,0,39,1,0.00,2,0,0,93826.63,0,1.0,0.0,0.0
4,850,0,43,2,125510.82,1,1,1,79084.10,0,0.0,0.0,1.0


In [63]:
# Train Test Split and Standardize the data

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

X = df.drop(columns=['Exited'], axis=1)
y = df['Exited']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)



In [64]:
# save the results to a binary file

with open('labelencoder_gender.pkl', 'wb') as f:
    pickle.dump(labelencoder_gender, f)

with open('onehotencoder_geography.pkl', 'wb') as f:
    pickle.dump(onehotencoder_geography, f)

with open('scaler.pkl', 'wb') as f:
    pickle.dump(scaler, f)


In [65]:
## ANN using Pytorch

# define the neural network architecture for binary classification
class ANNModel(nn.Module):
    def __init__(self, input_size):
        super(ANNModel, self).__init__()
        self.hidden1 = nn.Linear(input_size, 64)  # Input -> Hidden Layer 1
        self.hidden2 = nn.Linear(64, 32)         # Hidden Layer 1 -> Hidden Layer 2
        self.output = nn.Linear(32, 1)           # Hidden Layer 2 -> Output Layer
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        x = torch.relu(self.hidden1(x))
        x = torch.relu(self.hidden2(x))
        x = self.sigmoid(self.output(x))
        return x

# Instantiate the model
input_size = X_train.shape[1]
model = ANNModel(input_size)

# Binary Cross-Entropy Loss
criterion = nn.BCELoss()

# Optimizer (Adam)
optimizer = optim.Adam(model.parameters(), lr=0.01)


In [66]:
# Early Stopping
class EarlyStopping:
    def __init__(self, patience=15, verbose=False):
        self.patience = patience
        self.verbose = verbose
        self.best_loss = float('inf')
        self.counter = 0
        self.stop_training = False

    def __call__(self, val_loss, model):
        if val_loss < self.best_loss:
            self.best_loss = val_loss
            self.counter = 0
            self.best_model_weights = model.state_dict()  # Save best model
        else:
            self.counter += 1
            if self.verbose:
                print(f"EarlyStopping counter: {self.counter}/{self.patience}")
            if self.counter >= self.patience:
                self.stop_training = True

In [67]:
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train.to_numpy(), dtype=torch.float32).unsqueeze(1)  # Add a dimension for compatibility
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test.to_numpy(), dtype=torch.float32).unsqueeze(1)

# Create DataLoader
train_data = TensorDataset(X_train_tensor, y_train_tensor)
test_data = TensorDataset(X_test_tensor, y_test_tensor)

train_loader = DataLoader(train_data, batch_size=32, shuffle=True)
test_loader = DataLoader(test_data, batch_size=32, shuffle=False)


In [68]:
# TensorBoard logging
log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
writer = SummaryWriter(log_dir=log_dir)


In [ ]:
num_epochs = 50
early_stopping = EarlyStopping(patience=15, verbose=True)

for epoch in range(num_epochs):
    model.train()
    train_loss = 0.0

    for X_batch, y_batch in train_loader:
        optimizer.zero_grad()
        y_pred = model(X_batch)
        loss = criterion(y_pred, y_batch)
        loss.backward()
        optimizer.step()
        train_loss += loss.item()

    train_loss /= len(train_loader)

    # Validation
    model.eval()
    val_loss = 0.0
    with torch.no_grad():
        for X_batch, y_batch in test_loader:
            y_pred = model(X_batch)
            loss = criterion(y_pred, y_batch)
            val_loss += loss.item()
    val_loss /= len(test_loader)

    # Log to TensorBoard
    writer.add_scalar('Loss/train', train_loss, epoch)
    writer.add_scalar('Loss/validation', val_loss, epoch)

    print(f"Epoch {epoch+1}/{num_epochs}, Train Loss: {train_loss:.4f}, Val Loss: {val_loss:.4f}")

    # Early stopping
    early_stopping(val_loss, model)
    if early_stopping.stop_training:
        print("Early stopping triggered.")
        break

# Restore the best model
model.load_state_dict(early_stopping.best_model_weights)


Epoch 1/50, Train Loss: 0.4067, Val Loss: 0.3517
Epoch 2/50, Train Loss: 0.3580, Val Loss: 0.3494
Epoch 3/50, Train Loss: 0.3461, Val Loss: 0.3444
Epoch 4/50, Train Loss: 0.3444, Val Loss: 0.3440
Epoch 5/50, Train Loss: 0.3421, Val Loss: 0.3420
Epoch 6/50, Train Loss: 0.3401, Val Loss: 0.3447
EarlyStopping counter: 1/15
Epoch 7/50, Train Loss: 0.3365, Val Loss: 0.3528
EarlyStopping counter: 2/15
Epoch 8/50, Train Loss: 0.3320, Val Loss: 0.3457
EarlyStopping counter: 3/15
Epoch 9/50, Train Loss: 0.3320, Val Loss: 0.3424
EarlyStopping counter: 4/15
Epoch 10/50, Train Loss: 0.3289, Val Loss: 0.3462
EarlyStopping counter: 5/15
Epoch 11/50, Train Loss: 0.3276, Val Loss: 0.3494
EarlyStopping counter: 6/15
Epoch 12/50, Train Loss: 0.3280, Val Loss: 0.3422
EarlyStopping counter: 7/15
Epoch 13/50, Train Loss: 0.3226, Val Loss: 0.3618
EarlyStopping counter: 8/15
Epoch 14/50, Train Loss: 0.3228, Val Loss: 0.3452
EarlyStopping counter: 9/15
Epoch 15/50, Train Loss: 0.3204, Val Loss: 0.3590
EarlySt

<All keys matched successfully>

In [70]:
# Save PyTorch model in .pth format
torch.save(model.state_dict(), "model.pth")


In [78]:
# Predictions

import torch
import pickle
import pandas as pd
import numpy as np

# Load the PyTorch model
model = ANNModel(input_size=12)  # Update input_size to match the model
model.load_state_dict(torch.load("model.pth"))
model.eval()

# Load the scaler and encoders
with open('scaler.pkl', 'rb') as f:
    scaler = pickle.load(f)

with open('onehotencoder_geography.pkl', 'rb') as f:
    one_hot_encoder_geo = pickle.load(f)

with open('labelencoder_gender.pkl', 'rb') as f:
    label_encoder_gender = pickle.load(f)


C:\Users\hp\AppData\Local\Temp\ipykernel_15860\1057503147.py:10: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load("model.pth"))


In [52]:
# Provided input data
input_data = {
    'CreditScore': 600,
    'Geography': 'France',
    'Gender': 'Male',
    'Age': 40,
    'Tenure': 3,
    'Balance': 60000,
    'NumOfProducts': 2,
    'HasCrCard': 1,
    'IsActiveMember': 1,
    'EstimatedSalary': 50000
}




In [72]:
# Convert input data to DataFrame
input_df = pd.DataFrame([input_data])

# One-hot encode 'Geography'
geo_encoded = one_hot_encoder_geo.transform([input_df['Geography'].values]).toarray()
geo_encoded_df = pd.DataFrame(geo_encoded, columns=['Geography_France', 'Geography_Germany', 'Geography_Spain'])
input_df = pd.concat([input_df.drop(columns='Geography'), geo_encoded_df], axis=1)

# Encode 'Gender'
input_df['Gender'] = label_encoder_gender.transform(input_df['Gender'])

# Scale the input data
input_scaled = scaler.transform(input_df)
input_tensor = torch.tensor(input_scaled, dtype=torch.float32)

C:\Users\hp\anaconda3\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but OneHotEncoder was fitted with feature names
  warnings.warn(


In [73]:
# Make predictions with the PyTorch model
with torch.no_grad():
    predictions = model(input_tensor)
    prediction_prob = predictions.item()  # Extract the probability value

if prediction_prob > 0.5:
    print("The customer is likely to churn")
else:
    print("The customer is not likely to churn")

print(f"Prediction Probability: {prediction_prob:.4f}")


The customer is not likely to churn
Prediction Probability: 0.1108


In [74]:
from sklearn.metrics import accuracy_score

# Predictions on test data
model.eval()
with torch.no_grad():
    test_predictions = []
    for X_batch, y_batch in test_loader:
        y_pred = model(X_batch)
        y_pred_labels = (y_pred > 0.5).int()  # Threshold at 0.5
        test_predictions.extend(y_pred_labels.numpy())

# Convert to numpy for comparison
test_labels = y_test_tensor.numpy().flatten()
accuracy = accuracy_score(test_labels, test_predictions)
print(f"Model Accuracy on Test Data: {accuracy:.2%}")


Model Accuracy on Test Data: 84.90%


In [84]:
# Tensorboard display of logs
import torch
from torch.utils.tensorboard import SummaryWriter
from torch.utils import tensorboard
import datetime


In [79]:
dummy_input = torch.randn(1, input_size)  # Example input tensor matching model's input size
writer.add_graph(model, dummy_input)


In [80]:
writer.close()


In [ ]:
!tensorboard --logdir=logs/fit
